#### Zillow Prject Rought Draft Notebook

In [3]:
import math
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression, RFE

from env import user, host, password, sql_connect

In [4]:
##bulding SQL query to select the columns neccessary for project

##using a where clause with propertylandusetypeid to filter for
##data that I consider Single Unit Properties

sql_query = '''
select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt,       yearbuilt, taxamount, fips
from properties_2017
where propertylandusetypeid in (260, 261, 262, 263, 264, 265, 266, 273, 275, 276,       279); 
'''

In [5]:
## reading into a pandas dataframe and checking out the dataframe 

zillow_df = pd.read_sql(sql_query, sql_connect('zillow'))
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,563029.0,NaN,6773.34,6037.0
2,0.0,0.0,NaN,4265.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
4,0.0,0.0,NaN,10.0,NaN,NaN,6037.0


In [6]:
zillow_df.shape ##checking out the shape of your dataframe

(2709893, 7)

In [8]:
zillow_df.isnull().sum() ##looking at the sum of our null values

## given our data shape (row size) and the amount of null data is insignificant 
## to the amount of data we have. So we are going to drop the rows with 
## null values

bedroomcnt                         11
bathroomcnt                        23
calculatedfinishedsquarefeet    38198
taxvaluedollarcnt               31192
yearbuilt                       38257
taxamount                       16452
fips                                0
dtype: int64

In [9]:
zillow_df = zillow_df.dropna()  ## <-- drop the nulls
zillow_df.shape 

## we still have plenty of data to work with

(2637531, 7)

In [10]:
zillow_df.isnull().sum() 

## checking to see if nulls are gone

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [15]:
## we also are going to make sure there are no duplicates using the code
## below

zillow_df.drop_duplicates(subset = None, keep = 'first', inplace = False)
zillow_df.shape  ## we still have 

(2637531, 7)